Converts FITS file SEDs for $\alpha$ Cen and the Woods solar IDL file into appropriate txt files for VULCAN

In [10]:
%matplotlib qt

In [11]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from astropy.io import fits
from astropy import units as u
from scipy.io import readsav
from prettyplots import pretty_axis

### Functions ###

# Function to read and convert FITS data
def read_alfcen_data(filepath):
    data = fits.getdata(filepath, 1)
    return {
        'wavelength': data['wavelength'] * wave_conversion,
        'flux': data['totflux'] * flux_conversion
    }

# Convert to DataFrames
def to_df(data):
    return pd.DataFrame({'wavelength': data['wavelength'], 'flux': data['flux']})

In [20]:
# unit conversions
flux_conversion = (u.erg / u.s / u.cm**2 / u.AA).to(u.erg / u.s / u.cm**2 / u.nm)
wave_conversion = u.AA.to(u.nm)

# File paths
alfcen_files = {
    'peak': './alfCen_planetflux_peak_2024-6-17.fits',
    'min': './alfCen_planetflux_min_2024-6-17.fits',
}
woods_file = './atm/stellar_flux/solar-data.idlsav'


alfcen = {key: read_alfcen_data(path) for key, path in alfcen_files.items()}

# Load Woods data and convert units
woods_data = readsav(woods_file)
woods = {
    'wavelength': woods_data['wave'].flatten() * 10 * wave_conversion,
    'flux': woods_data['flux'].flatten() * 100 * flux_conversion
}

# Apply wavelength/flux masks and trim overlapping regions
woods_max_wave = woods['wavelength'][-1]
for key in alfcen:
    mask = (alfcen[key]['wavelength'] < woods_max_wave) & (alfcen[key]['flux'] > 1e-4)
    alfcen[key]['wavelength'] = alfcen[key]['wavelength'][mask]
    alfcen[key]['flux'] = alfcen[key]['flux'][mask]

woods_mask = woods['wavelength'] > alfcen['peak']['wavelength'][0]
woods['wavelength'] = woods['wavelength'][woods_mask]
woods['flux'] = woods['flux'][woods_mask]


fig,ax = plt.subplots(1,1,layout='constrained')
pretty_axis(ax)
ax.loglog(alfcen['peak']['wavelength'],alfcen['peak']['flux'],label=r'$\alpha$ Cen (peak)')
ax.loglog(alfcen['min']['wavelength'],alfcen['min']['flux'],label=r'$\alpha$ Cen (min)')
ax.loglog(woods['wavelength'],woods['flux'],label='Sun')
ax.set_xlabel('Wavelength [nm]',fontsize=18)
ax.set_ylabel(r'Flux [erg s$^{-1}$ cm$^{-2}$ nm$^{-1}$]',fontsize=18)
ax.legend(fontsize=14)

print('Bolometric fluxes')
ac_peak_bolo = np.trapezoid(x=alfcen['peak']['wavelength'],y=alfcen['peak']['flux'])
ac_min_bolo = np.trapezoid(x=alfcen['min']['wavelength'],y=alfcen['min']['flux'])
woods_bolo = np.trapezoid(x=woods['wavelength'],y=woods['flux'])

print('α Cen peak:\t%0.2f\tFraction of solar:\t%0.2f'%(ac_peak_bolo,ac_peak_bolo/woods_bolo))
print('α Cen min:\t%0.2f\tFraction of solar:\t%0.2f'%(ac_min_bolo,ac_min_bolo/woods_bolo))
print('Woods solar:\t',woods_bolo)

Bolometric fluxes
α Cen peak:	1261238.12	Fraction of solar:	0.96
α Cen min:	1263243.00	Fraction of solar:	0.96
Woods solar:	 1309836.2


In [21]:
dfs = {
    'alfcen_peak': to_df(alfcen['peak']),
    'alfcen_min': to_df(alfcen['min']),
    'woods': to_df(woods)
}

# Save to files
output_paths = {
    'alfcen_peak': './atm/stellar_flux/alf_cen_peak.txt',
    'alfcen_min': './atm/stellar_flux/alf_cen_min.txt',
    'woods': './atm/stellar_flux/woods_solar.txt'
}

for key, df in dfs.items():
    df.to_csv(output_paths[key], index=False, sep='\t')